# HW07 :  Aspect Tagging using RNN

ในหัวข้อนี้จะได้เรียนรู้การสร้างโมเดลสำหรับกำกับข้อมูลลำดับด้วย RNN โดยเริ่มต้นจากการทำ NER หลังจากนั้นจะได้ลองสร้างแบบจำลองเพื่อทำนาย Aspect ของข้อความโดยใช้ RNN


# Data Preparation


โดยคลังข้อมูลที่ใช้จะใช้ข้อมูล LST20

ดาวน์โหลดได้ที่ https://aiforthai.in.th/download.php?c=LST20

โดยสามารถดาวน์โหลดได้ตามขึ้นตอนดังนี้



In [ ]:
!pip install -U gdown

  Attempting uninstall: gdown
    Found existing installation: gdown 4.7.3
    Uninstalling gdown-4.7.3:
      Successfully uninstalled gdown-4.7.3


In [ ]:
#https://drive.google.com/file/d/1RQYB_ZGiCEMHNRlTN4vQgXK-9UifBmJQ/view?usp=drive_link

!gdown 1RQYB_ZGiCEMHNRlTN4vQgXK-9UifBmJQ

Downloading...
From: https://drive.google.com/uc?id=1RQYB_ZGiCEMHNRlTN4vQgXK-9UifBmJQ
To: /content/AIFORTHAI-LST20Corpus.tar.gz
100% 13.6M/13.6M [00:00<00:00, 109MB/s] 


In [ ]:
!mv AIFORTHAI-LST20Corpus.tar.gz LST20.tar.gz
!tar -xvf LST20.tar.gz

Streaming output truncated to the last 5000 lines.
LST20_Corpus/train/._T12371.txt
LST20_Corpus/train/T12371.txt
LST20_Corpus/train/._T11678.txt
LST20_Corpus/train/T11678.txt
LST20_Corpus/train/._T10566.txt
LST20_Corpus/train/T10566.txt
LST20_Corpus/train/._T00775.txt
LST20_Corpus/train/T00775.txt
LST20_Corpus/train/._T12417.txt
LST20_Corpus/train/T12417.txt
LST20_Corpus/train/._T12403.txt
LST20_Corpus/train/T12403.txt
LST20_Corpus/train/._T10214.txt
LST20_Corpus/train/T10214.txt
LST20_Corpus/train/._T01319.txt
LST20_Corpus/train/T01319.txt
LST20_Corpus/train/._T00761.txt
LST20_Corpus/train/T00761.txt
LST20_Corpus/train/._T01457.txt
LST20_Corpus/train/T01457.txt
LST20_Corpus/train/._T00991.txt
LST20_Corpus/train/T00991.txt
LST20_Corpus/train/._T11644.txt
LST20_Corpus/train/T11644.txt
LST20_Corpus/train/._T00749.txt
LST20_Corpus/train/T00749.txt
LST20_Corpus/train/._T13053.txt
LST20_Corpus/train/T13053.txt
LST20_Corpus/train/._T11122.txt
LST20_Corpus/train/T11122.txt
LST20_Corpus/train/

In [ ]:
!head -n 20 LST20_Corpus/eval/T01608.txt

สลด	VV	O	B_CLS
ผัว	NN	O	I_CLS
เมีย	NN	O	I_CLS
เฒ่า	NN	O	I_CLS
ถูก	AX	O	I_CLS
ไฟ	NN	O	I_CLS
ช็อต	VV	O	I_CLS
กอด	VV	O	I_CLS
กัน	AV	O	I_CLS
ตาย	VV	O	I_CLS
_	PU	O	I_CLS
สังเวย	VV	O	I_CLS
ฝน	NN	O	I_CLS
ตก	VV	O	I_CLS
-	PU	O	I_CLS
น้ำ	NN	O	I_CLS
ท่วม	VV	O	E_CLS

สอง	NU	O	B_CLS
สามี	NN	O	I_CLS


In [ ]:
def clean_data(fname):
    with open(fname.replace(".txt","_clean.txt"),"w") as fw:
        with open(fname) as fp:
            lines = fp.readlines()

            for line in lines:
                #print(line)

                if line.strip()=="":
                    fw.write("\n")
                else:
                    #print(line.split("\t"))
                    if len(line.split("\t"))==4:
                        fw.write(line)

In [ ]:
#create train/test/val data
!cat LST20_Corpus/train/T*.txt > LST20_Corpus/train.txt
!cat LST20_Corpus/eval/T*.txt > LST20_Corpus/val.txt
!cat LST20_Corpus/test/T*.txt > LST20_Corpus/test.txt


In [ ]:
clean_data("LST20_Corpus/train.txt")
clean_data("LST20_Corpus/val.txt")
clean_data("LST20_Corpus/test.txt")

In [ ]:
!head -n 10 LST20_Corpus/train_clean.txt

สุรยุทธ์	NN	B_PER	B_CLS
ยัน	VV	O	I_CLS
ปฏิเสธ	VV	O	I_CLS
ลงนาม	VV	O	I_CLS
_	PU	O	I_CLS
MOU	NN	O	I_CLS
_	PU	O	I_CLS
กับ	PS	O	I_CLS
อียู	NN	B_ORG	I_CLS
ไม่	NG	O	I_CLS


# CRF-based Thai NER

ในส่วนนี้เราจะสร้างแบบจำลอง NER สำหรับภาษาไทยโดยใช้คลังข้อมูล LST20 กัน

โดยจะอาศัยตัวอย่างโค้ดจาก

https://albertauyeung.github.io/2017/06/17/python-sequence-labelling-with-crf.html/

In [ ]:
!pip install python-crfsuite

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 8.2 MB/s eta 0:00:00


## เตรียมข้อมูล

โหลดข้อมูลให้ตรงตามรูปแบบ โดยแต่ละประโยคจะมีหน้าตา ดังนี้ [(w, pos, label),(w, pos, label),...]

In [ ]:
#[[('##', '##'),('The', 'AT'),('Fulton', 'NP-TL'),('&&', '&&')],[]]
def load_data(fname):
  data = []
  with open(fname) as fp:
      lines = fp.readlines()
      tmp_sen = []
      for line in lines:

          if line.strip()=="":
              if len(tmp_sen) > 0:
                  #tmp_sen.insert(0,('##','##'))
                  #tmp_sen.append(('&&','&&'))
                  data.append(tmp_sen)
                  tmp_sen = []

              continue

          _x = line.split('\t')

          tmp_sen.append((_x[0],_x[1],_x[2]))
  return data

In [ ]:
train_data=load_data('/content/LST20_Corpus/train_clean.txt')
test_data=load_data('/content/LST20_Corpus/test_clean.txt')

In [ ]:
train_data[:2]

[[('สุรยุทธ์', 'NN', 'B_PER'),
  ('ยัน', 'VV', 'O'),
  ('ปฏิเสธ', 'VV', 'O'),
  ('ลงนาม', 'VV', 'O'),
  ('_', 'PU', 'O'),
  ('MOU', 'NN', 'O'),
  ('_', 'PU', 'O'),
  ('กับ', 'PS', 'O'),
  ('อียู', 'NN', 'B_ORG'),
  ('ไม่', 'NG', 'O'),
  ('กระทบ', 'VV', 'O'),
  ('สัมพันธ์', 'NN', 'O')],
 [('1', 'NU', 'B_DTM'),
  ('_', 'PU', 'I_DTM'),
  ('กันยายน', 'NN', 'I_DTM'),
  ('_', 'PU', 'I_DTM'),
  ('2550', 'NU', 'E_DTM'),
  ('_', 'PU', 'O'),
  ('12:21', 'NU', 'B_DTM'),
  ('_', 'PU', 'I_DTM'),
  ('น.', 'CL', 'E_DTM')]]

## สร้าง Feature

โดยเราจะใช้ Feature ประมาณนี้


*   The word w itself (converted to lowercase for normalisation)
*   The prefix/suffix of w (e.g. -ion)
*   The words surrounding w, such as the previous and the next word
*   Whether w is in uppercase or lowercase
*   Whether w is a number, or contains digits
*   The POS tag of w, and those of the surrounding words
*   Whether w is or contains a special character (e.g. hypen, dollar sign)


In [ ]:
def word2features(doc, i):
    word = doc[i][0]
    postag = doc[i][1]

    # Common features for all words
    features = [
        'bias',
        'word.lower=' + word.lower(),
        'word.isupper=%s' % word.isupper(),
        'word.istitle=%s' % word.istitle(),
        'word.isdigit=%s' % word.isdigit(),
    ]

    # Features for words that are not
    # at the beginning of a document
    if i > 0:
        word1 = doc[i-1][0]
        postag1 = doc[i-1][1]

        features.extend([
            'prev_word.lower=' + word1.lower(),
            'prev_word.isupper=%s' % word1.isupper(),
            'prev_word.istitle=%s' % word1.istitle(),
            'prev_word.isdigit=%s' % word1.isdigit(),
        ])
    else:
        # Indicate that it is the 'beginning of a document'
        features.append('BOS')

    # Features for words that are not
    # at the end of a document
    if i < len(doc)-1:
        word1 = doc[i+1][0]
        postag1 = doc[i+1][1]
        #features.extend([])
        features.extend([
            'next_word.lower=' + word1.lower(),
            'next_word.isupper=%s' % word1.isupper(),
            'next_word.istitle=%s' % word1.istitle(),
            'next_word.isdigit=%s' % word1.isdigit(),
        ])
    else:
        # Indicate that it is the 'end of a document'
        features.append('EOS')

    return features

In [ ]:
from sklearn.model_selection import train_test_split

# A function for extracting features in documents
def extract_features(doc):
    return [word2features(doc, i) for i in range(len(doc))]

# A function fo generating the list of labels for each document
def get_labels(doc):
    return [label for (token, postag, label) in doc]

X_train = [extract_features(doc) for doc in train_data]
y_train = [get_labels(doc) for doc in train_data]

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
X_test = [extract_features(doc) for doc in test_data]
y_test = [get_labels(doc) for doc in test_data]

In [ ]:
type(X_train)

list

In [ ]:
import pycrfsuite
trainer = pycrfsuite.Trainer(verbose=True)

# Submit training data to the trainer
for xseq, yseq in zip(X_train, y_train):
    trainer.append(xseq, yseq)

# Set the parameters of the model
trainer.set_params({
    # coefficient for L1 penalty
    'c1': 0.1,

    # coefficient for L2 penalty
    'c2': 0.01,

    # maximum number of iterations
    'max_iterations': 50,

    # whether to include transitions that
    # are possible, but not observed
    'feature.possible_transitions': True
})

# Provide a file name as a parameter to the train function, such that
# the model will be saved to the file when training is finished
trainer.train('crf_ner.model')

Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 204799
Seconds required: 4.965

L-BFGS optimization
c1: 0.100000
c2: 0.010000
num_memories: 6
max_iterations: 50
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 3798291.811847
Feature norm: 1.000000
Error norm: 4011398.515198
Active features: 203948
Line search trials: 1
Line search step: 0.000000
Seconds required for this iteration: 26.483

***** Iteration #2 *****
Loss: 3557007.392080
Feature norm: 2.303659
Error norm: 1492466.345310
Active features: 196339
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 13.292

***** Iteration #3 *****
Loss: 3082290.760307
Feature norm: 1.972976
Error norm: 1266194.125857
Active features: 199096
Line search trials: 1
Line search step: 1.000000
Sec

KeyboardInterrupt: 

## ทดสอบและวัดประสิทธิภาพ

In [ ]:
tagger = pycrfsuite.Tagger()
tagger.open('crf_ner.model')
y_pred = [tagger.tag(xseq) for xseq in X_test]

# Let's take a look at a random sample in the testing set
i = 12
for x, y in zip(y_pred[i], [x[1].split("=")[1] for x in X_test[i]]):
    print("%s (%s)" % (y, x))

'''
The following will be printed:

sci-med (N)
life (N)
systems (N)
inc (N)
said (I)
its (I)
directors (I)
approved (I)
a (I)
previously (I)
...
'''

In [ ]:
import numpy as np
from sklearn.metrics import classification_report

# Convert the sequences of tags into a 1-dimensional array
_predictions = [tag for row in y_pred for tag in row]
_truths = [tag for row in y_test for tag in row]


except_tag = ['O'] #ไม่เอา tag "O"

In [ ]:
truths = []
predictions = []
for x,y in zip(_truths,_predictions):
  if x not in except_tag:
    truths.append(x)
    predictions.append(y)

In [ ]:
import numpy as np
from sklearn.metrics import classification_report

# Print out the classification report
print(classification_report(
    truths, predictions))

## RNN-based NER model

ในส่วนนี้เราจะเรียนรู้จากตัวอย่างโค้ดจาก
https://notebook.community/sameersingh/uci-statnlp/tutorials/rnn_examples

โดยจะลองรันชุดข้อมูลภาษาอังกฤษก่อน และลอง ปรับชุดข้อมูลเป็นภาษาไทย (LST20)

In [ ]:
!wget https://raw.githubusercontent.com/foxik/UD_English/master/en-ud-dev.conllu
!wget https://raw.githubusercontent.com/foxik/UD_English/master/en-ud-test.conllu
!wget https://raw.githubusercontent.com/foxik/UD_English/master/en-ud-train.conllu

--2024-01-26 07:44:49--  https://raw.githubusercontent.com/foxik/UD_English/master/en-ud-dev.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1191722 (1.1M) [text/plain]
Saving to: ‘en-ud-dev.conllu’

en-ud-dev.conllu    100%[===================>]   1.14M  --.-KB/s    in 0.03s   

2024-01-26 07:44:49 (35.1 MB/s) - ‘en-ud-dev.conllu’ saved [1191722/1191722]

--2024-01-26 07:44:49--  https://raw.githubusercontent.com/foxik/UD_English/master/en-ud-test.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1183830 (1.1M) [text/

In [ ]:
!head -n 5 en-ud-dev.conllu

head: cannot open 'en-ud-dev.conllu' for reading: No such file or directory


In [ ]:
#โค้ดส่วนนี้จะใช้สำหรับแปลง คำ ให้เป็น id

from collections import Counter

class Vocab(object):
    def __init__(self, iter, max_size=None, sos_token=None, eos_token=None, unk_token=None):
        """Initialize the vocabulary.
        Args:
            iter: An iterable which produces sequences of tokens used to update
                the vocabulary.
            max_size: (Optional) Maximum number of tokens in the vocabulary.
            sos_token: (Optional) Token denoting the start of a sequence.
            eos_token: (Optional) Token denoting the end of a sequence.
            unk_token: (Optional) Token denoting an unknown element in a
                sequence.
        """
        self.max_size = max_size
        self.pad_token = '<pad>'
        self.sos_token = sos_token
        self.eos_token = eos_token
        self.unk_token = unk_token

        # Add special tokens.
        id2word = [self.pad_token]
        if sos_token is not None:
            id2word.append(self.sos_token)
        if eos_token is not None:
            id2word.append(self.eos_token)
        if unk_token is not None:
            id2word.append(self.unk_token)

        # Update counter with token counts.
        counter = Counter()
        for x in iter:
            counter.update(x)

        # Extract lookup tables.
        if max_size is not None:
            counts = counter.most_common(max_size)
        else:
            counts = counter.items()
            counts = sorted(counts, key=lambda x: x[1], reverse=True)
        words = [x[0] for x in counts]
        id2word.extend(words)
        word2id = {x: i for i, x in enumerate(id2word)}

        self._id2word = id2word
        self._word2id = word2id

    def __len__(self):
        return len(self._id2word)

    def word2id(self, word):
        """Map a word in the vocabulary to its unique integer id.
        Args:
            word: Word to lookup.
        Returns:
            id: The integer id of the word being looked up.
        """
        if word in self._word2id:
            return self._word2id[word]
        elif self.unk_token is not None:
            return self._word2id[self.unk_token]
        else:
            raise KeyError('Word "%s" not in vocabulary.' % word)

    def id2word(self, id):
        """Map an integer id to its corresponding word in the vocabulary.
        Args:
            id: Integer id of the word being looked up.
        Returns:
            word: The corresponding word.
        """
        return self._id2word[id]

In [ ]:
import re
from torch.utils.data import Dataset


class Annotation(object):
    def __init__(self):
        """A helper object for storing annotation data."""
        self.tokens = []
        self.tags = []


class CoNLLDataset(Dataset):
    def __init__(self, fname):
        """Initializes the CoNLLDataset.
        Args:
            fname: The .conllu file to load data from.
        """
        self.fname = fname
        self.annotations = self.process_conll_file(fname)
        self.token_vocab = Vocab([x.tokens for x in self.annotations],
                                 unk_token='<unk>')
        self.pos_vocab = Vocab([x.tags for x in self.annotations],unk_token='<unk>')

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        annotation = self.annotations[idx]
        input = [self.token_vocab.word2id(x) for x in annotation.tokens]
        target = [self.pos_vocab.word2id(x) for x in annotation.tags]
        return input, target

    def process_conll_file(self, fname):
        # Read the entire file.
        with open(fname, 'r') as f:
            raw_text = f.read()
        # Split into chunks on blank lines.
        chunks = re.split(r'^\n', raw_text, flags=re.MULTILINE)
        # Process each chunk into an annotation.
        annotations = []
        for chunk in chunks:
            annotation = Annotation()
            lines = chunk.split('\n')
            # Iterate over all lines in the chunk.
            for line in lines:
                # If line is empty ignore it.
                if len(line)==0:
                    continue
                # If line is a commend ignore it.
                if line[0] == '#':
                    continue
                # Otherwise split on tabs and retrieve the token and the
                # POS tag fields.
                fields = line.split('\t')
                annotation.tokens.append(fields[1])  #<------ input ของโมเดล
                annotation.tags.append(fields[3])    #<------ output ของโมเดล อยากให้ทำนายอะไร

            if (len(annotation.tokens) > 0) and (len(annotation.tags) > 0):
                annotations.append(annotation)
        return annotations

In [ ]:
dataset = CoNLLDataset('en-ud-train.conllu')

In [ ]:
#ลองดูหน้าตาของข้อมูล

input, target = dataset[0]
print('Example input: %s\n' % input)
print('Example target: %s\n' % target)
print('Translated input: %s\n' % ' '.join(dataset.token_vocab.id2word(x) for x in input))
print('Translated target: %s\n' % ' '.join(dataset.pos_vocab.id2word(x) for x in target))

Example input: [262, 16, 5139, 45, 289, 689, 1123, 4138, 9863, 582, 16, 9864, 4, 3, 6722, 35, 3, 6723, 10, 3, 486, 8, 6724, 4, 740, 3, 2174, 1569, 2]

Example target: [7, 3, 7, 3, 8, 1, 2, 7, 7, 7, 3, 7, 3, 5, 1, 4, 5, 1, 4, 5, 1, 4, 7, 3, 4, 5, 8, 1, 3]

Translated input: Al - Zaman : American forces killed Shaikh Abdullah al - Ani , the preacher at the mosque in the town of Qaim , near the Syrian border .

Translated target: PROPN PUNCT PROPN PUNCT ADJ NOUN VERB PROPN PROPN PROPN PUNCT PROPN PUNCT DET NOUN ADP DET NOUN ADP DET NOUN ADP PROPN PUNCT ADP DET ADJ NOUN PUNCT



In [ ]:
import torch
from torch.autograd import Variable


def pad(sequences, max_length, pad_value=0):
    """Pads a list of sequences.
    Args:
        sequences: A list of sequences to be padded.
        max_length: The length to pad to.
        pad_value: The value used for padding.
    Returns:
        A list of padded sequences.
    """
    out = []
    for sequence in sequences:
        padded = sequence + [0]*(max_length - len(sequence))
        out.append(padded)
    return out


def collate_annotations(batch):
    """Function used to collate data returned by CoNLLDataset."""
    # Get inputs, targets, and lengths.
    inputs, targets = zip(*batch)
    lengths = [len(x) for x in inputs]
    # Sort by length.
    sort = sorted(zip(inputs, targets, lengths),
                  key=lambda x: x[2],
                  reverse=True)
    inputs, targets, lengths = zip(*sort)
    # Pad.
    max_length = max(lengths)
    inputs = pad(inputs, max_length)
    targets = pad(targets, max_length)
    # Transpose.
    inputs = list(map(list, zip(*inputs)))
    targets = list(map(list, zip(*targets)))
    # Convert to PyTorch variables.
    inputs = Variable(torch.LongTensor(inputs))
    targets = Variable(torch.LongTensor(targets))
    lengths = Variable(torch.LongTensor(lengths))
    if torch.cuda.is_available():
        inputs = inputs.cuda()
        targets = targets.cuda()
        lengths = lengths.cuda()
    return inputs, targets, lengths

In [ ]:
from torch.utils.data import DataLoader


for inputs, targets, lengths in DataLoader(dataset, batch_size=16, collate_fn=collate_annotations):
    print(inputs.shape)
    print(targets.data.shape)
    print(lengths.data.shape)

    # Usually we'd keep sampling batches, but here we'll just break
    break

torch.Size([36, 16])
torch.Size([36, 16])
torch.Size([16])


In [ ]:
from torch import nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

class Tagger(nn.Module):
    def __init__(self,
                 input_vocab_size,
                 output_vocab_size,
                 embedding_dim=64,
                 hidden_size=64,
                 bidirectional=True,num_layers=2):
        """Initializes the tagger.

        Args:
            input_vocab_size: Size of the input vocabulary.
            output_vocab_size: Size of the output vocabulary.
            embedding_dim: Dimension of the word embeddings.
            hidden_size: Number of units in each LSTM hidden layer.
            bidirectional: Whether or not to use a bidirectional rnn.
        """
        # Always do this!!!
        super(Tagger, self).__init__()

        # Store parameters
        self.input_vocab_size = input_vocab_size
        self.output_vocab_size = output_vocab_size
        self.embedding_dim = embedding_dim
        self.hidden_size = hidden_size
        self.bidirectional = bidirectional
        self.num_layers = num_layers
        # Define layers
        self.word_embeddings = nn.Embedding(input_vocab_size, embedding_dim,
                                            padding_idx=0)
        self.rnn = nn.GRU(embedding_dim, hidden_size,
                          bidirectional=bidirectional,num_layers=num_layers,
                          dropout=0.9)
        if bidirectional:
            self.fc = nn.Linear(2*hidden_size, output_vocab_size)
        else:
            self.fc = nn.Linear(hidden_size, output_vocab_size)
        self.activation = nn.LogSoftmax(dim=2)

    def forward(self, x, lengths=None, hidden=None):
        """Computes a forward pass of the language model.

        Args:
            x: A LongTensor w/ dimension [seq_len, batch_size].
            lengths: The lengths of the sequences in x.
            hidden: Hidden state to be fed into the lstm.

        Returns:
            net: Probability of the next word in the sequence.
            hidden: Hidden state of the lstm.
        """
        seq_len, batch_size = x.size()

        # If no hidden state is provided, then default to zeros.
        if hidden is None:
            if self.bidirectional:
                num_directions = 2
            else:
                num_directions = 1
            hidden = Variable(torch.zeros(num_directions*self.num_layers, batch_size, self.hidden_size))
            if torch.cuda.is_available():
                hidden = hidden.cuda()

        net = self.word_embeddings(x)
        # Pack before feeding into the RNN.
        if lengths is not None:
            lengths = lengths.data.view(-1).tolist()
            net = pack_padded_sequence(net, lengths)
        net, hidden = self.rnn(net, hidden)
        # Unpack after
        if lengths is not None:
            net, _ = pad_packed_sequence(net)
        net = self.fc(net)
        net = self.activation(net)

        return net, hidden

In [ ]:
import numpy as np

# Load datasets.
train_dataset = CoNLLDataset('en-ud-train.conllu')
dev_dataset = CoNLLDataset('en-ud-dev.conllu')

dev_dataset.token_vocab = train_dataset.token_vocab
dev_dataset.pos_vocab = train_dataset.pos_vocab

# Hyperparameters / constants.
input_vocab_size = len(train_dataset.token_vocab)
output_vocab_size = len(train_dataset.pos_vocab)
batch_size = 16
epochs = 5

In [ ]:
print(input_vocab_size,output_vocab_size)

19654 18


In [ ]:
# Initialize the model.
model = Tagger(input_vocab_size, output_vocab_size)
if torch.cuda.is_available():
    model = model.cuda()

# Loss function weights.
weight = torch.ones(output_vocab_size)
weight[0] = 0
if torch.cuda.is_available():
    weight = weight.cuda()

# Initialize loss function and optimizer.
loss_function = torch.nn.NLLLoss(weight)
optimizer = torch.optim.Adam(model.parameters())

# Main training loop.
data_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,
                         collate_fn=collate_annotations)
dev_loader = DataLoader(dev_dataset, batch_size=batch_size, shuffle=False,
                        collate_fn=collate_annotations)
losses = []
i = 0
min_losses = float('inf')
from tqdm import tqdm
for epoch in range(epochs):
    #print('epoch',epoch)
    for inputs, targets, lengths in tqdm(data_loader):
        optimizer.zero_grad()
        outputs, _ = model(inputs, lengths=lengths)

        outputs = outputs.view(-1, output_vocab_size)
        targets = targets.view(-1)

        loss = loss_function(outputs, targets)
        loss.backward()
        optimizer.step()

        losses.append(loss.item())
        if (i % 1) == 0:
            # Compute dev loss over entire dev set.
            # NOTE: This is expensive. In your work you may want to only use a
            # subset of the dev set.
            dev_losses = []
            for inputs, targets, lengths in dev_loader:
                outputs, _ = model(inputs, lengths=lengths)
                outputs = outputs.view(-1, output_vocab_size)
                targets = targets.view(-1)
                loss = loss_function(outputs, targets)
                dev_losses.append(loss.item())
            avg_train_loss = np.mean(losses)
            avg_dev_loss = np.mean(dev_losses)
            losses = []
            #print('Iteration %i - Train Loss: %0.6f - Dev Loss: %0.6f' % (i, avg_train_loss, avg_dev_loss), end='\r')

            #เพิ่มโค้ด เพื่อให้บันทึกโมเดลรอบที่ loss ต่ำสุด
            print(avg_dev_loss)
            if avg_dev_loss < min_losses:
              torch.save(model, 'pos_tragger_best.pt')
              min_losses = avg_dev_loss

            torch.save(model, 'pos_tagger.pt')
        i += 1

torch.save(model, 'pos_tagger.final.pt')

  0%|          | 0/6 [00:00<?, ?it/s]


KeyError: 'Word "B_COMMENT_NEG" not in vocabulary.'

### Model Evaluation

In [ ]:
state_dict = torch.load('pos_tagger.pt', map_location=torch.device('cpu'))

#create model for inference
model = Tagger(input_vocab_size, output_vocab_size)
model.load_state_dict(state_dict.state_dict())
model.eval()

In [ ]:
print(model)

In [ ]:
model.cuda()

In [ ]:
y_true = []
y_pred = []

for inputs, targets, lengths in test_loader:
    outputs, _ = model(inputs, lengths=lengths)
    _, preds = torch.max(outputs, dim=2)
    targets = targets.view(-1)
    preds = preds.view(-1)
    if torch.cuda.is_available():
        targets = targets.cpu()
        preds = preds.cpu()
    y_true.append(targets.data.numpy())
    y_pred.append(preds.data.numpy())

# Stack into numpy arrays
y_true = np.concatenate(y_true)
y_pred = np.concatenate(y_pred)

# Compute accuracy
acc = np.mean(y_true[y_true != 0] == y_pred[y_true != 0])
print('Accuracy - %0.6f\n' % acc)

# Evaluate f1-score
from sklearn.metrics import f1_score
score = f1_score(y_true, y_pred, average=None)
print('F1-scores:\n')
for label, score in zip(dev_dataset.pos_vocab._id2word[1:], score[1:]):
    print('%s - %0.6f' % (label, score))

Accuracy - 0.303151

F1-scores:

<unk> - 0.482930
O - 0.162791
I_PRODUCTQUALITY_POS - 0.008230
I_APPEARANCE_POS - 0.000000
I_DELIVERYTIME_POS - 0.000000
I_PRODUCTQUALITY_NEG - 0.027397
I_PACKAGING_POS - 0.000000
I_STORESERVICE_POS - 0.000000
I_PRICE_POS - 0.000000
I_DELIVERYSERVICE_POS - 0.000000
I_PACKAGING_NEG - 0.000000
I_APPEARANCE_NEG - 0.000000
B_PRODUCTQUALITY_POS - 0.000000
I_STORESERVICE_NEG - 0.000000
B_DELIVERYTIME_POS - 0.000000
B_APPEARANCE_POS - 0.000000
I_DELIVERYTIME_NEG - 0.000000
I_DELIVERYSERVICE_NEG - 0.000000
B_PRICE_POS - 0.000000
B_PRODUCTQUALITY_NEG - 0.000000
B_PACKAGING_POS - 0.000000
I_IMPRESSED_NEG - 0.000000
B_STORESERVICE_POS - 0.000000
B_DELIVERYSERVICE_POS - 0.000000
B_APPEARANCE_NEG - 0.000000
I_IMPRESSED_POS - 0.000000
I_PRICE_NEG - 0.000000
B_STORESERVICE_NEG - 0.000000
I_DELIVERY_SERVICE_NEG - 0.000000
B_PACKAGING_NEG - 0.000000
I_DELIVERY_SERVICE_POS - 0.000000
B_DELIVERYTIME_NEG - 0.000000
B_DELIVERYSERVICE_NEG - 0.000000
I_PROMOTION_POS - 0.000000

# Assignment

1) ให้สร้างแบบจำลองสำหรับทำ Aspect sentiment Analysis จากข้อมูลที่นีกศึกษาช่วยกันรวมตั้งแต่ต้นเทอม
Aspect sentiment Analysis เป็นการกำกับว่าช่วงไหนของข้อความที่่เกี่ยวข้องกับประเด็นต่างๆ เช่น ราคาสินค้า การจัดส่ง เป็นต้น โดนนักศึกษาจะต้องสร้างโมเดลดังนี้

  **Model 1:** GRU 128*1L

  **Model 2:** GRU 128*3L




2) ให้ทำการวัดผลแสดงค่า Precision Recall F1-score แล้วทำการอภิปรายผลลัพธ์


### Data Preparation

In [ ]:
#https://drive.google.com/file/d/1Ee6pfAaleE3NOZA4lB96-gI7Rh9ngzFc/view?usp=sharing
!gdown 1Ee6pfAaleE3NOZA4lB96-gI7Rh9ngzFc
#https://drive.google.com/file/d/1-H8wHcD54QjFlHsOZnW9Ad7rB3-87Cb3/view?usp=sharing
!gdown 1-H8wHcD54QjFlHsOZnW9Ad7rB3-87Cb3

#https://drive.google.com/file/d/1-Awk_LoTYD4rrclerMFBogbC8jH6GPU-/view?usp=sharing
!gdown 1-Awk_LoTYD4rrclerMFBogbC8jH6GPU-

Downloading...
From: https://drive.google.com/uc?id=1Ee6pfAaleE3NOZA4lB96-gI7Rh9ngzFc
To: /content/train.txt
100% 297k/297k [00:00<00:00, 3.79MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-H8wHcD54QjFlHsOZnW9Ad7rB3-87Cb3
To: /content/test.txt
100% 36.0k/36.0k [00:00<00:00, 93.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-Awk_LoTYD4rrclerMFBogbC8jH6GPU-
To: /content/valid.txt
100% 37.3k/37.3k [00:00<00:00, 79.4MB/s]


### Create Dataset in CoNLL format

In [ ]:
def write_conll_file(sentences, file_path):
    with open(file_path, 'w', encoding='utf-8') as file:
        for sentence in sentences:
            for word, label in sentence:
                file.write(f"{word}\t{label}\n")
            file.write('\n')

In [ ]:
!pip install pythainlp

In [ ]:
from pythainlp.tokenize import word_tokenize
import re

def parse_data(input_file,output_file):
    sentences = []
    current_sentence = []

    with open(input_file, 'r', encoding='utf-8') as file:
        for line in file:
            line = line.strip()
            if not line:  # Empty line indicates the end of a sentence
                if current_sentence:
                    sentences.append(current_sentence)
                    current_sentence = []
            else:
                parts = line.strip().split('\t')
                if len(parts) >= 2:  # Ensure the line has at least two columns (word and label)
                    word = word_tokenize(parts[0])
                    if parts[1]=='O':
                      label = ["O"]*len(word)
                    else:
                      label = ["I_"+parts[1]]*len(word)
                    if label[0]!='O':
                      label[0] = 'B_'+parts[1]

                    for w,label in zip(word,label):
                      current_sentence.append([w.strip() if w.strip()!='' else '_', label])


    if current_sentence:
        sentences.append(current_sentence)

    write_conll_file(sentences, output_file)
    #print(sentences[0])


In [ ]:
parse_data('train.txt','train.conllu')
parse_data('test.txt','test.conllu')
parse_data('valid.txt','valid.conllu')

In [ ]:
!head -n 100 'train.conllu'
!head -n 100 'test.conllu'
!head -n 100 'valid.conllu'

In [ ]:
import re
from torch.utils.data import Dataset


class Annotation(object):
    def __init__(self):
        """A helper object for storing annotation data."""
        self.tokens = []
        self.tags = []


class CoNLLDataset(Dataset):
    def __init__(self, fname):
        """Initializes the CoNLLDataset.
        Args:
            fname: The .conllu file to load data from.
        """
        self.fname = fname
        self.annotations = self.process_conll_file(fname)
        self.token_vocab = Vocab([x.tokens for x in self.annotations],
                                 unk_token='<unk>')
        self.pos_vocab = Vocab([x.tags for x in self.annotations],
                               unk_token='<unk>')

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        annotation = self.annotations[idx]
        input = [self.token_vocab.word2id(x) for x in annotation.tokens]
        target = [self.pos_vocab.word2id(x) for x in annotation.tags]
        return input, target

    def process_conll_file(self, fname):
        # Read the entire file.
        with open(fname, 'r') as f:
            raw_text = f.read()
        # Split into chunks on blank lines.
        chunks = re.split(r'^\n', raw_text, flags=re.MULTILINE)
        # Process each chunk into an annotation.
        annotations = []
        for chunk in chunks:
            annotation = Annotation()
            lines = chunk.split('\n')
            # Iterate over all lines in the chunk.
            for line in lines:
                # If line is empty ignore it.
                if len(line)==0:
                    continue
                # If line is a commend ignore it.
                if line[0] == '#':
                    continue
                # Otherwise split on tabs and retrieve the token and the
                # POS tag fields.
                fields = line.split('\t')
                annotation.tokens.append(fields[0])  #<------ input ของโมเดล
                annotation.tags.append(fields[1])    #<------ output ของโมเดล อยากให้ทำนายอะไร

            if (len(annotation.tokens) > 0) and (len(annotation.tags) > 0):
                annotations.append(annotation)
        return annotations

In [ ]:
import numpy as np

# Load datasets.
train_dataset = CoNLLDataset('train.conllu')
dev_dataset = CoNLLDataset('valid.conllu')
test_dataset = CoNLLDataset('test.conllu')

dev_dataset.token_vocab = train_dataset.token_vocab
dev_dataset.pos_vocab = train_dataset.pos_vocab

test_dataset.token_vocab = train_dataset.token_vocab
test_dataset.pos_vocab = train_dataset.pos_vocab

# Hyperparameters / constants.
input_vocab_size = len(train_dataset.token_vocab)
output_vocab_size = len(train_dataset.pos_vocab)
batch_size = 128
epochs = 100

In [ ]:
# Initialize the model.
model = Tagger(input_vocab_size, output_vocab_size,num_layers=1,hidden_size=128)
if torch.cuda.is_available():
    model = model.cuda()

# Loss function weights.
weight = torch.ones(output_vocab_size)
weight[0] = 0
if torch.cuda.is_available():
    weight = weight.cuda()

# Initialize loss function and optimizer.
loss_function = torch.nn.NLLLoss(weight)
optimizer = torch.optim.Adam(model.parameters())

# Main training loop.
data_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,
                         collate_fn=collate_annotations)
dev_loader = DataLoader(dev_dataset, batch_size=batch_size, shuffle=False,
                        collate_fn=collate_annotations)

test_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,
                         collate_fn=collate_annotations)
test_loader = DataLoader(dev_dataset, batch_size=batch_size, shuffle=False,
                        collate_fn=collate_annotations)


losses = []
i = 0
min_losses = float('inf')
from tqdm import tqdm
for epoch in range(epochs):
    print('epoch',epoch)
    for inputs, targets, lengths in tqdm(data_loader):
        optimizer.zero_grad()
        outputs, _ = model(inputs, lengths=lengths)

        outputs = outputs.view(-1, output_vocab_size)
        targets = targets.view(-1)

        loss = loss_function(outputs, targets)
        loss.backward()
        optimizer.step()

        losses.append(loss.item())
        if (i % 1) == 0:
            # Compute dev loss over entire dev set.
            # NOTE: This is expensive. In your work you may want to only use a
            # subset of the dev set.
            dev_losses = []
            for inputs, targets, lengths in dev_loader:
                outputs, _ = model(inputs, lengths=lengths)
                outputs = outputs.view(-1, output_vocab_size)
                targets = targets.view(-1)
                loss = loss_function(outputs, targets)
                dev_losses.append(loss.item())
            avg_train_loss = np.mean(losses)
            avg_dev_loss = np.mean(dev_losses)
            losses = []
            print('Iteration %i - Train Loss: %0.6f - Dev Loss: %0.6f' % (i, avg_train_loss, avg_dev_loss), end='\r')

            #เพิ่มโค้ด เพื่อให้บันทึกโมเดลรอบที่ loss ต่ำสุด
            print(avg_dev_loss)
            if avg_dev_loss < min_losses:
              torch.save(model, 'pos_tragger_best.pt')
              min_losses = avg_dev_loss

            torch.save(model, 'pos_tagger.pt')
        i += 1

torch.save(model, 'pos_tagger.final.pt')

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:82: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.9 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


epoch 0


 17%|█▋        | 1/6 [00:00<00:00,  8.83it/s]

4.186506271362305
4.139051914215088


 50%|█████     | 3/6 [00:00<00:00, 12.78it/s]

...

3.6311569213867188
3.640068769454956
3.655407428741455


 67%|██████▋   | 4/6 [00:00<00:00, 18.96it/s]

3.6647799015045166


100%|██████████| 6/6 [00:00<00:00, 20.02it/s]


3.669595718383789
3.675045967102051


In [ ]:
state_dict = torch.load('pos_tagger.pt', map_location=torch.device('cpu'))

#create model for inference
model = Tagger(input_vocab_size, output_vocab_size,num_layers=1,hidden_size=128)
model.load_state_dict(state_dict.state_dict())
model.eval()

Tagger(
  (word_embeddings): Embedding(2382, 64, padding_idx=0)
  (rnn): GRU(64, 128, dropout=0.9, bidirectional=True)
  (fc): Linear(in_features=256, out_features=69, bias=True)
  (activation): LogSoftmax(dim=2)
)

In [ ]:
model.cuda()


Tagger(
  (word_embeddings): Embedding(2382, 64, padding_idx=0)
  (rnn): GRU(64, 128, dropout=0.9, bidirectional=True)
  (fc): Linear(in_features=256, out_features=69, bias=True)
  (activation): LogSoftmax(dim=2)
)

In [ ]:
y_true = []
y_pred = []

for inputs, targets, lengths in test_loader:
    outputs, _ = model(inputs, lengths=lengths)
    _, preds = torch.max(outputs, dim=2)
    targets = targets.view(-1)
    preds = preds.view(-1)
    if torch.cuda.is_available():
        targets = targets.cpu()
        preds = preds.cpu()
    y_true.append(targets.data.numpy())
    y_pred.append(preds.data.numpy())

# Stack into numpy arrays
y_true = np.concatenate(y_true)
y_pred = np.concatenate(y_pred)

# Compute accuracy
acc = np.mean(y_true[y_true != 0] == y_pred[y_true != 0])
print('Accuracy - %0.6f\n' % acc)

# Evaluate f1-score
from sklearn.metrics import f1_score
score = f1_score(y_true, y_pred, average=None)
print('F1-scores:\n')
for label, score in zip(dev_dataset.pos_vocab._id2word[1:], score[1:]):
    print('%s - %0.6f' % (label, score))

Accuracy - 0.345973

F1-scores:

<unk> - 0.000000
O - 0.466194
I_PRODUCTQUALITY_POS - 0.327820
I_APPEARANCE_POS - 0.229947
I_DELIVERYTIME_POS - 0.578171
I_PRODUCTQUALITY_NEG - 0.207547
I_PACKAGING_POS - 0.525140
I_STORESERVICE_POS - 0.275449
I_PRICE_POS - 0.315068
I_DELIVERYSERVICE_POS - 0.323077
I_PACKAGING_NEG - 0.062016
I_APPEARANCE_NEG - 0.001021
B_PRODUCTQUALITY_POS - 0.317460
I_STORESERVICE_NEG - 0.248447
B_DELIVERYTIME_POS - 0.580645
B_APPEARANCE_POS - 0.324324
I_DELIVERYTIME_NEG - 0.114286
I_DELIVERYSERVICE_NEG - 0.000000
B_PRICE_POS - 0.390244
B_PRODUCTQUALITY_NEG - 0.105263
B_PACKAGING_POS - 0.482759
I_IMPRESSED_NEG - 0.000000
B_STORESERVICE_POS - 0.303030
B_DELIVERYSERVICE_POS - 0.307692
B_APPEARANCE_NEG - 0.111111
I_IMPRESSED_POS - 0.000000
I_PRICE_NEG - 0.242424
B_STORESERVICE_NEG - 0.153846
I_DELIVERY_SERVICE_NEG - 0.000000
B_PACKAGING_NEG - 0.000000
I_DELIVERY_SERVICE_POS - 0.000000
B_DELIVERYTIME_NEG - 0.222222
B_DELIVERYSERVICE_NEG - 0.000000
I_PROMOTION_POS - 0.000000

# model 2

---



In [ ]:
parse_data('train.txt','train.conllu')
parse_data('test.txt','test.conllu')
parse_data('valid.txt','valid.conllu')

In [ ]:
!head -n 100 'train.conllu'
!head -n 100 'test.conllu'
!head -n 100 'valid.conllu'

In [ ]:
import re
from torch.utils.data import Dataset


class Annotation(object):
    def __init__(self):
        """A helper object for storing annotation data."""
        self.tokens = []
        self.tags = []


class CoNLLDataset(Dataset):
    def __init__(self, fname):
        """Initializes the CoNLLDataset.
        Args:
            fname: The .conllu file to load data from.
        """
        self.fname = fname
        self.annotations = self.process_conll_file(fname)
        self.token_vocab = Vocab([x.tokens for x in self.annotations],
                                 unk_token='<unk>')
        self.pos_vocab = Vocab([x.tags for x in self.annotations],
                               unk_token='<unk>')

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        annotation = self.annotations[idx]
        input = [self.token_vocab.word2id(x) for x in annotation.tokens]
        target = [self.pos_vocab.word2id(x) for x in annotation.tags]
        return input, target

    def process_conll_file(self, fname):
        # Read the entire file.
        with open(fname, 'r') as f:
            raw_text = f.read()
        # Split into chunks on blank lines.
        chunks = re.split(r'^\n', raw_text, flags=re.MULTILINE)
        # Process each chunk into an annotation.
        annotations = []
        for chunk in chunks:
            annotation = Annotation()
            lines = chunk.split('\n')
            # Iterate over all lines in the chunk.
            for line in lines:
                # If line is empty ignore it.
                if len(line)==0:
                    continue
                # If line is a commend ignore it.
                if line[0] == '#':
                    continue
                # Otherwise split on tabs and retrieve the token and the
                # POS tag fields.
                fields = line.split('\t')
                annotation.tokens.append(fields[0])  #<------ input ของโมเดล
                annotation.tags.append(fields[1])    #<------ output ของโมเดล อยากให้ทำนายอะไร

            if (len(annotation.tokens) > 0) and (len(annotation.tags) > 0):
                annotations.append(annotation)
        return annotations

In [ ]:
import numpy as np

# Load datasets.
train_dataset = CoNLLDataset('train.conllu')
dev_dataset = CoNLLDataset('valid.conllu')
test_dataset = CoNLLDataset('test.conllu')

dev_dataset.token_vocab = train_dataset.token_vocab
dev_dataset.pos_vocab = train_dataset.pos_vocab

test_dataset.token_vocab = train_dataset.token_vocab
test_dataset.pos_vocab = train_dataset.pos_vocab

# Hyperparameters / constants.
input_vocab_size = len(train_dataset.token_vocab)
output_vocab_size = len(train_dataset.pos_vocab)
batch_size = 128
epochs = 100

In [ ]:
# Initialize the model.
model = Tagger(input_vocab_size, output_vocab_size,num_layers = 3, hidden_size= 128)
if torch.cuda.is_available():
    model = model.cuda()

# Loss function weights.
weight = torch.ones(output_vocab_size)
weight[0] = 0
if torch.cuda.is_available():
    weight = weight.cuda()

# Initialize loss function and optimizer.
loss_function = torch.nn.NLLLoss(weight)
optimizer = torch.optim.Adam(model.parameters())

# Main training loop.
data_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,
                         collate_fn=collate_annotations)
dev_loader = DataLoader(dev_dataset, batch_size=batch_size, shuffle=False,
                        collate_fn=collate_annotations)

test_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,
                         collate_fn=collate_annotations)
test_loader = DataLoader(dev_dataset, batch_size=batch_size, shuffle=False,
                        collate_fn=collate_annotations)


losses = []
i = 0
min_losses = float('inf')
from tqdm import tqdm
for epoch in range(epochs):
    print('epoch',epoch)
    for inputs, targets, lengths in tqdm(data_loader):
        optimizer.zero_grad()
        outputs, _ = model(inputs, lengths=lengths)

        outputs = outputs.view(-1, output_vocab_size)
        targets = targets.view(-1)

        loss = loss_function(outputs, targets)
        loss.backward()
        optimizer.step()

        losses.append(loss.item())
        if (i % 1) == 0:
            # Compute dev loss over entire dev set.
            # NOTE: This is expensive. In your work you may want to only use a
            # subset of the dev set.
            dev_losses = []
            for inputs, targets, lengths in dev_loader:
                outputs, _ = model(inputs, lengths=lengths)
                outputs = outputs.view(-1, output_vocab_size)
                targets = targets.view(-1)
                loss = loss_function(outputs, targets)
                dev_losses.append(loss.item())
            avg_train_loss = np.mean(losses)
            avg_dev_loss = np.mean(dev_losses)
            losses = []
            print('Iteration %i - Train Loss: %0.6f - Dev Loss: %0.6f' % (i, avg_train_loss, avg_dev_loss), end='\r')

            #เพิ่มโค้ด เพื่อให้บันทึกโมเดลรอบที่ loss ต่ำสุด
            print(avg_dev_loss)
            if avg_dev_loss < min_losses:
              torch.save(model, 'pos_tragger_best.pt')
              min_losses = avg_dev_loss

            torch.save(model, 'pos_tagger.pt')
        i += 1

torch.save(model, 'pos_tagger.final.pt')

epoch 0


 17%|█▋        | 1/6 [00:00<00:00,  8.34it/s]

4.207900047302246


 33%|███▎      | 2/6 [00:00<00:00,  8.57it/s]

4.150801658630371
4.080548286437988


...

 67%|██████▋   | 4/6 [00:00<00:00, 13.40it/s]

3.568584680557251


100%|██████████| 6/6 [00:00<00:00, 14.26it/s]

3.639111280441284
3.6371443271636963


100%|██████████| 6/6 [00:00<00:00, 13.74it/s]


In [ ]:
state_dict = torch.load('pos_tagger.pt', map_location=torch.device('cpu'))

#create model for inference
model = Tagger(input_vocab_size, output_vocab_size,num_layers = 3, hidden_size= 128)
model.load_state_dict(state_dict.state_dict())
model.eval()

Tagger(
  (word_embeddings): Embedding(2382, 64, padding_idx=0)
  (rnn): GRU(64, 128, num_layers=3, dropout=0.9, bidirectional=True)
  (fc): Linear(in_features=256, out_features=69, bias=True)
  (activation): LogSoftmax(dim=2)
)

In [ ]:
model.cuda()

Tagger(
  (word_embeddings): Embedding(2382, 64, padding_idx=0)
  (rnn): GRU(64, 128, num_layers=3, dropout=0.9, bidirectional=True)
  (fc): Linear(in_features=256, out_features=69, bias=True)
  (activation): LogSoftmax(dim=2)
)

In [ ]:
y_true = []
y_pred = []

for inputs, targets, lengths in dev_loader:
    outputs, _ = model(inputs, lengths=lengths)
    _, preds = torch.max(outputs, dim=2)
    targets = targets.view(-1)
    preds = preds.view(-1)
    if torch.cuda.is_available():
        targets = targets.cpu()
        preds = preds.cpu()
    y_true.append(targets.data.numpy())
    y_pred.append(preds.data.numpy())

# Stack into numpy arrays
y_true = np.concatenate(y_true)
y_pred = np.concatenate(y_pred)

# Compute accuracy
acc = np.mean(y_true[y_true != 0] == y_pred[y_true != 0])
print('Accuracy - %0.6f\n' % acc)

# Evaluate f1-score
from sklearn.metrics import f1_score
score = f1_score(y_true, y_pred, average=None)
print('F1-scores:\n')
for label, score in zip(dev_dataset.pos_vocab._id2word[1:], score[1:]):
    print('%s - %0.6f' % (label, score))

Accuracy - 0.277718

F1-scores:

<unk> - 0.000000
O - 0.340187
I_PRODUCTQUALITY_POS - 0.291933
I_APPEARANCE_POS - 0.239278
I_DELIVERYTIME_POS - 0.521739
I_PRODUCTQUALITY_NEG - 0.204239
I_PACKAGING_POS - 0.455764
I_STORESERVICE_POS - 0.234637
I_PRICE_POS - 0.395722
I_DELIVERYSERVICE_POS - 0.198830
I_PACKAGING_NEG - 0.291667
I_APPEARANCE_NEG - 0.125000
B_PRODUCTQUALITY_POS - 0.003064
I_STORESERVICE_NEG - 0.061069
B_DELIVERYTIME_POS - 0.440000
B_APPEARANCE_POS - 0.187500
I_DELIVERYTIME_NEG - 0.222222
I_DELIVERYSERVICE_NEG - 0.000000
B_PRICE_POS - 0.254545
B_PRODUCTQUALITY_NEG - 0.125000
B_PACKAGING_POS - 0.321429
I_IMPRESSED_NEG - 0.000000
B_STORESERVICE_POS - 0.000000
B_DELIVERYSERVICE_POS - 0.076923
B_APPEARANCE_NEG - 0.000000
I_IMPRESSED_POS - 0.000000
I_PRICE_NEG - 0.000000
B_STORESERVICE_NEG - 0.000000
I_DELIVERY_SERVICE_NEG - 0.000000
B_PACKAGING_NEG - 0.000000
I_DELIVERY_SERVICE_POS - 0.000000
B_DELIVERYTIME_NEG - 0.285714
B_DELIVERYSERVICE_NEG - 0.000000
I_PROMOTION_POS - 0.000000

**ลองทำเพิ่มเติม**

https://github.com/bentrevett/pytorch-pos-tagging

https://github.com/yoseflaw/nerindo

https://notebook.community/sameersingh/uci-statnlp/tutorials/rnn_examples